# Smoothing
-----------

This notebook shows how to use `dysh` to smooth a spectrum.   We will use data from the Position-Switch example.

In [ ]:
import os
import wget
import astropy.units as u
from dysh.fits.gbtfitsload import GBTFITSLoad

## We use the FITS file already used in the Position-Switch example


In [ ]:
filename = "TGBT21A_501_11.raw.vegas.fits"
if not os.path.isfile(filename):
    url = f"http://www.gb.nrao.edu/dysh/example_data/onoff-L/data/{filename}"
    print(f"Downloading {filename}")
    wget.download(url,out=filename)
    print(f"\nRetrieved {filename}")
else:
    print(f"{filename} already downloaded")

In [ ]:
sdfits = GBTFITSLoad(filename)

## Get a time-averaged spectrum at the highest resolution


In [ ]:
psscan = sdfits.getps(scan=152, ifnum=0, plnum=0)
ta = psscan.timeaverage(weights='tsys')
ta.plot(xaxis_unit="chan",yaxis_unit="mK",ymin=-100,ymax=500, grid=True)

## Smooth

In [ ]:
ts1 = ta.smooth('gaussian',32)
ts1.plot(xaxis_unit="chan",yaxis_unit="mK",ymin=-100,ymax=500,grid=True)

In [ ]:
ts2 = ta.smooth('box', 320)
ts2.plot(xaxis_unit="chan",yaxis_unit="mK",ymin=-100,ymax=500, grid=True)

In [ ]:
ta.plot(xaxis_unit="km/s",yaxis_unit="mK",ymin=-100,ymax=500)

In [ ]:
ts2.plot(xaxis_unit="km/s",yaxis_unit="mK",ymin=-100,ymax=500)